# Phase separation
#### This notebook takes a pair of phase image unwrapped in Digital Micrograph and separates the electrostatic and magnetic phase contributions. 
#### In this notebook the images are aligned, 
#### warped to remove first order aberations, 
#### smoothed and rebinned, 
#### unwrapping errors are masked, 
#### masks defining confidence and material extent are created.

## 0. Setup
### Imports and defining file locations

In [1]:
%matplotlib qt

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import pickle

In [3]:
import fpd
import hyperspy.api as hs
import skimage.filters as skfl

In [4]:
#load editable developement pyramid version
import sys
import os
pyramid_dev_version_path=os.path.abspath(r"D:\Programmes\Git\empyre-patched\empyre")
sys.path.append(pyramid_dev_version_path)
#print(sys.path)
import pyramid as pr
pr

<module 'pyramid' from 'D:\\Programmes\\Git\\empyre-patched\\empyre\\pyramid\\__init__.py'>

In [5]:
#Code by A.S.
import pyramidas.alignment as pa
import pyramidas.util as pu
import pyramidas.reconstruction as pre

In [6]:
import hyperspy
hyperspy.__version__


'1.5.2'

In [7]:
#check what files are available
os.listdir('.')

['+0',
 '+20',
 '+30',
 '-10',
 '-20',
 '-30',
 '-40',
 '-50',
 '-60',
 '.ipynb_checkpoints',
 'data_series_unrefined.pickle',
 'Figure_3.tif',
 'v3 phi 0 phase separation.ipynb',
 'v3 phi 0 tilt series alignment.ipynb']

In [8]:
folders = ['+0',
 '+20',
 '+30',
 '-10',
 '-20',
 '-30',
 '-40',
 '-50',
 '-60']

#Tilt for each projection
x_tilts = [float(folder) for folder in folders] # deg

#filenames before and after flipping the sample
f_name1="side 1.dm4"
f_name2="side 2.dm4"

#Fringe spacing is used to find the nyquist frequency and maximum resolution
fringe_spacing = 2.9 #nm


In [9]:
#initiallise storage
no_projections=len(folders)
datas=[None]*no_projections

print(no_projections)

9


<a id='index_cell'></a>
## 1. Aligning phase images
### Run this section once for each tilt angle.
### Manual input is required to increment 'i', and to varity if alignment was successfull.

### 1.1 Importing files

In [10]:
#select which folder to process. i=0 is first folder in list
i = 0
print(f"Working on projection at {x_tilts[i]:.1f} deg tilt")

Working on projection at 0.0 deg tilt


In [11]:
#file paths

#original images
f_path1=folders[i]+'\\'+f_name1
f_path2=folders[i]+'\\'+f_name2

# file paths for saving
transform_path=f_path2[:-len(f_name2)]+"transform.pickle"
mag_phase_path=f_path1[:-len(f_name1)]+"mag_phase.tif"
elec_phase_path=f_path1[:-len(f_name1)]+"elec_phase.tif"
data_path=f_path1[:-len(f_name1)]+"data.pickle" #for saving results

In [12]:
#load dm files
s1_orig=hs.load(f_path1)
s2_orig=hs.load(f_path2)

print("loaded",f_path1,"and",f_path2,"\n")

s1, s2, a_spacing = pa.equalise_hspy_signals(s1_orig, s2_orig, fringe_spacing, plot_original = False, plot_cropped = True)

loaded +0\side 1.dm4 and +0\side 2.dm4 

Original image side 1 <Signal2D, title: side 1, dimensions: (|1855, 1919)>
Original image side 2 <Signal2D, title: side 2, dimensions: (|1855, 1919)>
Original pixel spacing = 0.64057 nm

*****

Rebinning by 2.0
Rebinned image side 1 <Signal2D, title: , dimensions: (|927, 959)>
Rebinned image side 2 <Signal2D, title: , dimensions: (|927, 959)>
Rebinned pixel spacing = 1.2811 nm


### 1.2 Aligning phase images

In [13]:
#set up image cropping

interactive = True # whether an interactive cropping UI will be used

#start the UI if it is used
if interactive:
    # make interactive RoI selector
    img=s1.data+s2.data
    s=hs.signals.Signal2D(img)
    s.metadata.General.title = "Select edge detection area"
    s.plot()
    left,right,top,bottom=s.axes_manager.signal_extent
    rect=hs.roi.RectangularROI(right*0.1,bottom*0.1,right*0.9,bottom*0.9)
    roi2D = rect.interactive(s) 
    
else:
    indexes = (116.0, 838.0, 235.0, 784.0)
    left,right,top,bottom=[int(x) for x in indexes]

In [14]:
if interactive:
    print("Adjust RoI now!")

Adjust RoI now!


In [15]:
if interactive:
    indexes = roi2D.axes_manager.signal_extent
    left,right,top,bottom=[int(x) for x in indexes]
    print("left, right, top, bottom", roi2D.axes_manager.signal_extent)

roi_joint = np.full(s1.data.shape, False)
roi_joint[top:bottom, left:right] = True

left, right, top, bottom (272.0, 818.0, 243.0, 757.0)


In [16]:
# identify sample edges
sigma=3 #increase to remove noise, but keep low so edges are preserved
high_threshold=3.2 #increase until only one edge remains
hole_filling_radius = 1 #improves smoothness of masked holes but slows down execution

#fill masked holes to avoid discontinuities
img1 = pa.fill_raw_dm_image(s1.data, erode_width=3, radius=hole_filling_radius)
img2 = pa.fill_raw_dm_image(s2.data, erode_width=3, radius=hole_filling_radius)

edges1,edges2 = pa.identify_edges(img1, img2, sigma=sigma, high_threshold=high_threshold, roi=roi_joint)

In [17]:
#measure affline transform between the two images
image_warped = True
optimise =  True

trans_meas = pa.measure_orb_transform(edges1,edges2, image_warped=image_warped, 
                                   optimise=optimise, trans_path=transform_path, roi=roi_joint)

Number of keypoints (left, right): 1000 1000
Number of matches: 146
Number of inliers: 24

nrmse
-----
initial:   0.7050
unwarped:  0.6500
optimised: 0.5304

Transformation
translation = [ -7.09190219 -42.62406692] *( dx,dy)
roation = 1.2171319410538985 deg, clockwise
scale = (0.9778124091610081, 0.9931817100710685) * (sx,sy)
shear -0.06699496783378463
Transformation saved as: +0\transform.pickle


In [18]:
#apply transform to image 2
s2_tr = s2.copy()
s2_tr.data = pa.apply_image_trans(s2.data, trans_meas)

#calculate the magnetic and electrostatic images
pm_mag=(s1-s2_tr)/2*(1) #-1 to account for wrong sideband being used in phase unwrapping.
pm_el=(s1+s2_tr)/2*(1)
pm_mag.metadata.General.title="magnetic phase, xtilt = %.1f"%(x_tilts[i])
pm_el.metadata.General.title="electrostatic phase, xtilt = %.1f"%(x_tilts[i])

pm_mag.plot()
pm_el.plot()

#save the magnetic phase image
img=Image.fromarray(pm_mag.data.astype('f4')) 
img.save(mag_phase_path)
print("Magnetic phase saved as:",mag_phase_path)

#and save electrostatic phase image
img=Image.fromarray(pm_el.data.astype('f4')) 
img.save(elec_phase_path)
print("Electrostatic phase saved as:",elec_phase_path)

Magnetic phase saved as: +0\mag_phase.tif
Electrostatic phase saved as: +0\elec_phase.tif


### 1.3 Refining phase components before reconstruction

In [199]:
#span fill correction by cropping
cropping=slice(None,None)

In [207]:
#Make rebinned, smooth phasemaps
sigma_mag_rec = 1 #sigma of gaussian filter before magnetisation reconstruction 
median_footprint=np.ones((9,9))
rebin_factor_rec = 8 # to make total number of voxels less than 10^6
phase_img_type='f4'
erode_depth=3 #number of pixels to remove from edge
fill_convolution_radius=2 #smoothness of filled holes


#Define mask showing which pixels are wrong.
wrong1=np.where(s1.data==0, True, False) 
wrong2=np.where(s2_tr.data==0, True, False)
wrong=np.logical_or(wrong1, wrong2)

#get transformed pixel spacing
hs_data=pm_mag.rebin(scale=(rebin_factor_rec,rebin_factor_rec))
pix_spacing = hs_data.axes_manager.signal_axes[0].scale # nm

phase_images = [pm_mag.data, pm_el.data]
phase_images = [im[cropping,cropping] for im in phase_images]
wrong = wrong[cropping,cropping]
processed_phase_images=[]
for img in phase_images:
    
    #the edge most pixels are wrong, hence erode the edges
    img, wrong = pu.erode_masked_image(img, wrong, radius=erode_depth, wrong_pixel_value=0) 
    
    #fill in holes and other free space with smooth extensions of the edges
    img, temp = pu.fill_masked_image(img, wrong, radius=fill_convolution_radius)
    
    #filter and rebin
    img = skfl.median(img, selem = median_footprint, behavior="ndimage") #remove difference errors and hot pixels
    img = skfl.gaussian(img, sigma=sigma_mag_rec) #smooth to remove high spatial frequency noise
    img = pu.rebin_img(img, rebin_factor=rebin_factor_rec) #reduce pixel number to reduce memory requirements
    img = img.astype(phase_img_type)
    processed_phase_images.append(img)
    
mag_phase, el_phase = processed_phase_images

#adjust confidence array to match
confidence=np.where(wrong,0,1)
confidence = pu.rebin_img(confidence, rebin_factor=rebin_factor_rec)
confidence = np.where(confidence>0.9, 1, 0)
confidence = confidence.astype(phase_img_type)

#highlight wrong areas
outside_mask = pa.get_space_around_mask(confidence<1)
#el_phase[outside_mask] = 0 #keep holes inside the object filled as that improves mask creation but remove unkown mask regions

#save and display
Image.fromarray(mag_phase).save(mag_phase_path+"_smoothed_rebinned.tif")
print("Image shapes:", mag_phase.shape, el_phase.shape, confidence.shape)
print("Image dtype:", mag_phase.dtype)
print("Pixel spacing:", pix_spacing, "nm")
pu.matshow_n([mag_phase,el_phase,confidence],["Magnetic phase", "Mean inner potential", "Confidence"])

Image shapes: (120, 116) (120, 116) (120, 116)
Image dtype: float32
Pixel spacing: 10.249117851257324 nm


### 1.4 Defining masks

In [208]:
#Normalise mean inner potential and define a mask.
mask_threshold=-3 #rad. where the wire starts if mean inner potential vacccum phase is 0.
residual_threshold = 2 #to avoid outliers in the background fit

#remove electrostatic phase ramp
(fit, inliers, n) = fpd.ransac_tools.ransac_im_fit(el_phase, 
                                                   residual_threshold=residual_threshold, mask = confidence < 1,
                                                   mode=2, plot=True)

# Make mean inner potential vaccum phase == 0.
el_flat = el_phase - fit
#el_flat[outside_mask]=0

# Define mask and remove false pixels
mask=np.where(el_flat<mask_threshold, True, False)
#mask[outside_mask] = False

print("2D mask shape:", mask.shape)
print("Mask threshold:", mask_threshold)
#check if threshold is appropriate
pu.matshow_n([el_flat, mask],[elec_phase_path[:-4]+"mask selector", elec_phase_path[:-4]+"mask"], save=True)


2D mask shape: (120, 116)
Mask threshold: -3


### 1.5 Saving results

In [202]:
#create data storage object for reconstruction and save it

xtilt=x_tilts[i]
save_data=True

print("i:",i)
print("xtilt:", xtilt)

datas[i]=pa.make_projection_data_simple(mag_phase, mask, confidence, xtilt=xtilt, pix_spacing=pix_spacing, save_data=save_data, data_path=data_path)

i: 8
xtilt: -60.0
Reconstruction voxel number: 13920
Pixel size: 10.2491 nm
3d reconstructions dimensions: (1, 120, 116)
starting projector calculation
projector calculation finished
Data saved as: -60\data.pickle


### 1.6 2D reconstruction to check correctness

In [32]:
#quick reconstruction

#simulation might be wrong if axis is tilted??

magdata_rec, cost_fun = pre.reconstruct_from_phasemaps_simple(datas[i], verbose=False, max_iter=200)

### Hyperlink
[Back to index defining cell](#index_cell)

#### End of notebook. Once phase images have been aligned for all tilt angles, see alignment notebook to form a tilt series.